In [4]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

from ray.tune import ExperimentAnalysis

notebook_path = os.getcwd()
sys.path.append(os.path.abspath(os.path.join(notebook_path, '..')))

from utils.constants import TRAIN_RESULTS_PATH
%matplotlib inline

In [ ]:
experiments_info = [
    {
        "containers": 6,
        "legend": "IMPALA-6",
        "algorithm": "IMPALA",
        "trial": "IMPALA_SimEdgeEnv_c3861_00000_0_2024-10-13_14-12-14"},
    {
        "containers": 12,
        "legend": "IMPALA-12",
        "algorithm": "IMPALA",
        "trial": "IMPALA_SimEdgeEnv_da1af_00000_0_2024-10-13_12-54-07"
    },
    {
        "containers": 18,
        "legend": "IMPALA-18",
        "algorithm": "IMPALA",
        "trial": "IMPALA_SimEdgeEnv_7f87e_00000_0_2024-10-14_20-14-13"
    }
]

Hello World!
